# DATA 603 PCS2 - ML
# Project - Telecom Customer Churn Prediction
# Hettiarachige Tissera

## Load Libraries

In [1]:
# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np


import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
import os 

import pandas as pd

pd.options.display.max_columns = None

# graph wizard
from graphviz import Source
from sklearn.tree import export_graphviz

# importing random forest classifier from assemble module
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# import the metrics class
from sklearn import metrics

# Importing GridSearchCV for parameter tuning
from sklearn.model_selection import  GridSearchCV

from sklearn import neighbors
from sklearn.naive_bayes import GaussianNB


from sklearn.ensemble import BaggingClassifier

from sklearn.model_selection import cross_val_score

In [ ]:
# import pandas
import pandas as pd

# Importing data spliting
from sklearn.model_selection import train_test_split

# import the metrics class
from sklearn import metrics
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Importing GridSearchCV for parameter tuning
from sklearn.model_selection import  GridSearchCV

# Importing classifiers
from sklearn import neighbors
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier

# Bagging 
from sklearn.ensemble import BaggingClassifier

from sklearn.model_selection import cross_val_score

## Load Telecom Dataset

In [2]:
df = pd.read_csv('Telco-Customer-Churn.csv')

## Exploratory Data Analysis (EDA)

In [3]:
# Load first five rows
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [4]:
print('No of Rows and Columns in the data frame')
df.shape

No of Rows and Columns in the data frame


(7043, 21)

In [5]:
# Display Data types
print ('Data Types')
df.dtypes

Data Types


customerID           object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

## Data Preparation

#### Converting all the Categorical values to Numerical data

In [11]:
replacements = {'No': 0,
               'Yes': 1,
               'Female': 0,
               'Male': 1,
               'No internet service' : 0,
               'No phone service' : 0}
                
df.replace(replacements, inplace = True)

#### Checking for NULL Values

In [13]:
# 'Total Charges Columns has blank values. converting them to NULL values'
df['TotalCharges']=df['TotalCharges'].replace(' ', np.nan)
df['TotalCharges'] = pd.to_numeric(df.TotalCharges, errors='coerce')

In [14]:
# Checking for Null values in the data set
df.isnull().sum()

customerID           0
gender               0
SeniorCitizen        0
Partner              0
Dependents           0
tenure               0
PhoneService         0
MultipleLines        0
InternetService      0
OnlineSecurity       0
OnlineBackup         0
DeviceProtection     0
TechSupport          0
StreamingTV          0
StreamingMovies      0
Contract             0
PaperlessBilling     0
PaymentMethod        0
MonthlyCharges       0
TotalCharges        11
Churn                0
dtype: int64

In [15]:
# Dropping NULL values
df = df.dropna()

In [16]:
# Checking for Null values in the data set
df.isnull().sum()

customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

In [17]:
# Drop columsn without any prediction value, Customer ID 
df=df.drop(columns='customerID', axis=1)
df.shape

(7032, 20)

#### One Hot Encoding the three Categorical Columns

In [19]:
print(df['InternetService'].unique())

['DSL' 'Fiber optic' 0]


In [20]:
print(df['Contract'].unique())

['Month-to-month' 'One year' 'Two year']


In [21]:
print(df['PaymentMethod'].unique())

['Electronic check' 'Mailed check' 'Bank transfer (automatic)'
 'Credit card (automatic)']


In [22]:
# SElecting the Categorical columns
categorical_cols = ['InternetService', 'Contract', 'PaymentMethod']
print(categorical_cols)

['InternetService', 'Contract', 'PaymentMethod']


In [23]:
one_hot_encoded_data = pd.get_dummies(df, columns = ['InternetService', 'PaymentMethod', 'Contract'])

In [25]:
df=one_hot_encoded_data

#### Checking the categorical parameter count

In [26]:
categorical_variables = ['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure',
       'PhoneService', 'MultipleLines', 'OnlineSecurity',
       'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV',
       'StreamingMovies', 'PaperlessBilling', 'Churn']
df[categorical_variables].apply(pd.Series.value_counts)

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,PaperlessBilling,Churn
0,3483.0,5890.0,3639.0,4933.0,NaN,680.0,4065.0,5017.0,4607.0,4614.0,4992.0,4329.0,4301.0,2864.0,5163.0
1,3549.0,1142.0,3393.0,2099.0,613.0,6352.0,2967.0,2015.0,2425.0,2418.0,2040.0,2703.0,2731.0,4168.0,1869.0
2,NaN,NaN,NaN,NaN,238.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,200.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,176.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,NaN,NaN,NaN,NaN,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
69,NaN,NaN,NaN,NaN,95.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
70,NaN,NaN,NaN,NaN,119.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71,NaN,NaN,NaN,NaN,170.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
# Converting all data point to floats
df.astype(float)

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,PaperlessBilling,MonthlyCharges,TotalCharges,Churn,InternetService_0,InternetService_DSL,InternetService_Fiber optic,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,Contract_Month-to-month,Contract_One year,Contract_Two year
0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,29.85,29.85,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1,1.0,0.0,0.0,0.0,34.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,56.95,1889.50,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
2,1.0,0.0,0.0,0.0,2.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,53.85,108.15,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
3,1.0,0.0,0.0,0.0,45.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,42.30,1840.75,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,70.70,151.65,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,1.0,0.0,1.0,1.0,24.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,84.80,1990.50,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
7039,0.0,0.0,1.0,1.0,72.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,103.20,7362.90,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
7040,0.0,0.0,1.0,1.0,11.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,29.60,346.45,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
7041,1.0,1.0,1.0,0.0,4.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,74.40,306.60,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0


In [28]:
features = list(df)
print( 'List of features used for Prediction are : ')
print([features])

List of features used for Prediction are : 
[['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure', 'PhoneService', 'MultipleLines', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'PaperlessBilling', 'MonthlyCharges', 'TotalCharges', 'Churn', 'InternetService_0', 'InternetService_DSL', 'InternetService_Fiber optic', 'PaymentMethod_Bank transfer (automatic)', 'PaymentMethod_Credit card (automatic)', 'PaymentMethod_Electronic check', 'PaymentMethod_Mailed check', 'Contract_Month-to-month', 'Contract_One year', 'Contract_Two year']]


#### Checking Class Imbalance


In [30]:
total_Clients = df.shape[0]
total_Churned_Clients = (np.sum(df['Churn'] == 1)/total_Clients)*100
total_Active_Clients = (np.sum(df['Churn'] == 0)/total_Clients)*100
print("Churned Clients", round(total_Churned_Clients,2),"%")
print("Active Clients", round(total_Active_Clients,2),"%")

Churned Clients 26.58 %
Active Clients 73.42 %


#### Converting the Data to Numpy Array

In [31]:
y = np.array(df['Churn'])

In [32]:
df_temp=df.drop(columns='Churn', axis=1)
X = np.array(df_temp)

#### Scale the Data

In [33]:
# import module
from sklearn.preprocessing import MinMaxScaler

X=MinMaxScaler().fit(X).transform(X)

# Splitting the Data - Train and Test

In [36]:
## Splitting the data to train and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123, stratify=y)

In [37]:
print('Training Features Shape:', X_train.shape)
print('Training Labels Shape:', y_train.shape)
print('Testing Features Shape:', X_test.shape)
print('Testing Labels Shape:', y_test.shape)

Training Features Shape: (4922, 26)
Training Labels Shape: (4922,)
Testing Features Shape: (2110, 26)
Testing Labels Shape: (2110,)


In [38]:
# Creating data frame to store accuracy values
df_accuracy = pd.DataFrame(columns=['Classifier', 'Accuracy Training data CV=5', 'Accuracy Testing data'])

#  Gaussian Classifier 

In [39]:
# Guassian Classifier
clf = GaussianNB()

# Grid Search CV to find best parameters
parameters = {'var_smoothing': np.logspace(0,-9, num=100)}

Grid_search_CV = GridSearchCV(estimator=clf, 
                 param_grid=parameters,  
                 scoring='accuracy',
                 cv = 5,
                 n_jobs = -1)

# Fitting the training data
Grid_search_CV.fit(X_train, y_train)

Grid_search_CV.best_params_

{'var_smoothing': 0.03511191734215131}

In [40]:
# Accuracy of training data
print("Accuracy for Training Data : %0.4s%% [%s]"% (Grid_search_CV.best_score_*100,clf))

#Predict the response for test dataset
y_pred = Grid_search_CV.predict(X_test)

# using metrics module for accuracy calculation
accuracy = accuracy_score(y_test, y_pred)*100
print("Accuracy for Testing Data : %0.4s%% [%s]"% (accuracy,clf))

Accuracy for Training Data : 74.4% [GaussianNB()]
Accuracy for Testing Data : 75.4% [GaussianNB()]


In [41]:
target_names = ['Churned', 'Not Churned']
print(classification_report(y_test, y_pred, target_names=target_names))

              precision    recall  f1-score   support

     Churned       0.91      0.74      0.82      1549
 Not Churned       0.53      0.78      0.63       561

    accuracy                           0.75      2110
   macro avg       0.72      0.76      0.72      2110
weighted avg       0.80      0.75      0.77      2110



In [42]:
# Updating accuracy table 
df_accuracy.loc[len(df_accuracy.index)] = [clf, Grid_search_CV.best_score_*100, accuracy] 

#### Ensemble Learning - Bagging

In [43]:
# Guassian Classifier
clf = GaussianNB()

# Bagging Classifier
bag = BaggingClassifier(base_estimator=clf,
                        n_estimators=30, 
                        max_samples=1.0, 
                        max_features=1.0, 
                        bootstrap=True, 
                        bootstrap_features=False, 
                        n_jobs=1, 
                        random_state=1)

In [44]:
# fit the bagging regressor 
bag = bag.fit(X_train, y_train) 

# Cross Validation with Traning data
scores=cross_val_score(estimator=bag, X=X_train,y=y_train,cv=5,scoring=None)

In [45]:
# Accuracy of training data
print("Accuracy for Training Data : %0.4s%% [%s]"% (scores.mean()*100,bag))

#Predict the response for test dataset
y_pred = bag.predict(X_test)

# using metrics module for accuracy calculation
accuracy = accuracy_score(y_test, y_pred)*100
print("Accuracy for Testing Data : %0.4s%% [%s] "% (accuracy,bag))

Accuracy for Training Data : 74.4% [BaggingClassifier(base_estimator=GaussianNB(), n_estimators=30, n_jobs=1,
                  random_state=1)]
Accuracy for Testing Data : 75.3% [BaggingClassifier(base_estimator=GaussianNB(), n_estimators=30, n_jobs=1,
                  random_state=1)] 


In [46]:
# Updating accuracy table 
df_accuracy.loc[len(df_accuracy.index)] = ['BaggingClassifier(base_estimator=GaussianNB()' , scores.mean()*100, accuracy] 

# Random Forest Classifier

In [48]:
# Random Forest Classifier
clf = RandomForestClassifier() 

# Grid Search CV to find best parameters
parameters = {
                'max_features': ['auto', 'sqrt', 'log2', None], 
                'max_depth': [2,3, 4],
                'criterion': ['gini', 'entropy'],
                'bootstrap': [True, False],
                'n_estimators': [2, 5, 10, 15, 20]
              }

Grid_search_CV = GridSearchCV(estimator=clf, 
                 param_grid=parameters,  
                 scoring='accuracy',
                 cv = 5,
                 n_jobs = -1)

# Fitting the training data
Grid_search_CV.fit(X_train, y_train)

Grid_search_CV.best_params_

{'bootstrap': True,
 'criterion': 'entropy',
 'max_depth': 4,
 'max_features': 'sqrt',
 'n_estimators': 15}

In [49]:
# Accuracy of training data
print("Accuracy for Training Data : %0.4s%% [%s]"% (Grid_search_CV.best_score_*100,clf))

#Predict the response for test dataset
y_pred = Grid_search_CV.predict(X_test)

# using metrics module for accuracy calculation
accuracy = accuracy_score(y_test, y_pred)*100
print("Accuracy for Testing Data : %0.4s%% [%s]"% (accuracy,clf))

Accuracy for Training Data : 79.9% [RandomForestClassifier()]
Accuracy for Testing Data : 78.8% [RandomForestClassifier()]


In [50]:
target_names = ['Churned', 'Not Churned']
print(classification_report(y_test, y_pred, target_names=target_names))

              precision    recall  f1-score   support

     Churned       0.80      0.95      0.87      1549
 Not Churned       0.70      0.35      0.47       561

    accuracy                           0.79      2110
   macro avg       0.75      0.65      0.67      2110
weighted avg       0.78      0.79      0.76      2110



In [51]:
# Updating accuracy table 
df_accuracy.loc[len(df_accuracy.index)] = [clf, Grid_search_CV.best_score_*100, accuracy] 

### Ensemble Learning - Bagging

In [79]:
# Random Forest Classifier
clf = RandomForestClassifier() 

# Bagging Classifier
bag = BaggingClassifier(base_estimator=clf,
                        n_estimators=30, 
                        max_samples=1.0, 
                        max_features=1.0, 
                        bootstrap=True, 
                        bootstrap_features=False, 
                        n_jobs=1, 
                        random_state=1)

In [80]:
# fit the bagging regressor 
bag = bag.fit(X_train, y_train) 

# Cross Validation with Traning data
scores=cross_val_score(estimator=bag, X=X_train,y=y_train,cv=5,scoring=None)

In [81]:
# Accuracy of training data
print("Accuracy for Training Data : %0.4s%% [%s]"% (scores.mean()*100,bag))

#Predict the response for test dataset
y_pred = bag.predict(X_test)

# using metrics module for accuracy calculation
accuracy = accuracy_score(y_test, y_pred)*100
print("Accuracy for Testing Data : %0.4s%% [%s] "% (accuracy,bag))

Accuracy for Training Data : 79.6% [BaggingClassifier(base_estimator=RandomForestClassifier(max_depth=4),
                  n_estimators=30, n_jobs=1, random_state=1)]
Accuracy for Testing Data : 78.3% [BaggingClassifier(base_estimator=RandomForestClassifier(max_depth=4),
                  n_estimators=30, n_jobs=1, random_state=1)] 


In [55]:
# Updating accuracy table 
df_accuracy.loc[len(df_accuracy.index)] = ['BaggingClassifier(base_estimator=RandomForestClassifier()' , scores.mean()*100, accuracy] 

# KNN Nearest Neighbours 

In [72]:
# KNN Classifier
clf = neighbors.KNeighborsClassifier()

# Grid Search CV to find best parameters
parameters = {'n_neighbors':[2,3,4,5,6,7,8,9, 12, 15]}

Grid_search_CV = GridSearchCV(estimator=clf, 
                 param_grid=parameters,  
                 scoring=None,
                 cv = 5,
                 n_jobs = -1)
#scoring='accuracy',
# Fitting the training data
Grid_search_CV.fit(X_train, y_train)

Grid_search_CV.best_params_

{'n_neighbors': 12}

In [58]:
# Accuracy of training data
print("Accuracy for Training Data : %0.4s%% [%s]"% (Grid_search_CV.best_score_*100,clf))

#Predict the response for test dataset
y_pred = Grid_search_CV.predict(X_test)

# using metrics module for accuracy calculation
accuracy = accuracy_score(y_test, y_pred)*100
print("Accuracy for Testing Data : %0.4s%% [%s]"% (accuracy,clf))

Accuracy for Training Data : 77.9% [KNeighborsClassifier()]
Accuracy for Testing Data : 77.4% [KNeighborsClassifier()]


In [59]:
target_names = ['Churned', 'Not Churned']
print(classification_report(y_test, y_pred, target_names=target_names))

              precision    recall  f1-score   support

     Churned       0.82      0.89      0.85      1549
 Not Churned       0.60      0.46      0.52       561

    accuracy                           0.77      2110
   macro avg       0.71      0.67      0.69      2110
weighted avg       0.76      0.77      0.76      2110



In [60]:
# Updating accuracy table 
df_accuracy.loc[len(df_accuracy.index)] = [clf, Grid_search_CV.best_score_*100, accuracy] 

### Ensemble Learning - Bagging

In [73]:
# KNN Classifier
clf = neighbors.KNeighborsClassifier(n_neighbors= 12)

#Bagging Classifier
bag = BaggingClassifier(base_estimator=clf,
                        n_estimators=30, 
                        max_samples=1.0, 
                        max_features=1.0, 
                        bootstrap=True, 
                        bootstrap_features=False, 
                        n_jobs=1, 
                        random_state=1)

In [74]:
# fit the bagging regressor 
bag = bag.fit(X_train, y_train) 

# Cross Validation with Traning data
scores=cross_val_score(estimator=bag, X=X_train,y=y_train,cv=5,scoring=None)

In [75]:
# Accuracy of training data
print("Accuracy for Training Data : %0.4s%% [%s]"% (scores.mean()*100,bag))

#Predict the response for test dataset
y_pred = bag.predict(X_test)

# using metrics module for accuracy calculation
accuracy = accuracy_score(y_test, y_pred)*100
print("Accuracy for Testing Data : %0.4s%% [%s] "% (accuracy,bag))

Accuracy for Training Data : 76.4% [BaggingClassifier(base_estimator=KNeighborsClassifier(), n_estimators=30,
                  n_jobs=1, random_state=1)]
Accuracy for Testing Data : 76.8% [BaggingClassifier(base_estimator=KNeighborsClassifier(), n_estimators=30,
                  n_jobs=1, random_state=1)] 


In [65]:
# Updating accuracy table 
df_accuracy.loc[len(df_accuracy.index)] = ['BaggingClassifier(base_estimator=KNeighborsClassifier()' , scores.mean()*100, accuracy] 

In [66]:
df_accuracy

,Classifier,Accuracy Training data CV=5,Accuracy Testing data
0,GaussianNB(),74.441047,75.497630
1,BaggingClassifier(base_estimator=GaussianNB(),74.481697,75.355450
2,RandomForestClassifier(),79.906545,78.815166
3,BaggingClassifier(base_estimator=RandomForestC...,79.723515,79.715640
4,KNeighborsClassifier(),77.996719,77.488152
5,BaggingClassifier(base_estimator=KNeighborsCla...,76.452602,76.824645
